## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

### This notebook will be used for the 2nd Peer-graded Assignment of the Data Science Capstone course.

## Question 1

Lets load required libraries

In [1]:
#importing librairies
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

### 1. Dataframe creation from Wikipedia web scraping

In [2]:
#importing more librairies
# import the library we use to open URLs

import urllib.request

In [3]:
# importing the table 

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)


In [4]:
pip install BeautifulSoup4

     |████████████████████████████████| 122kB 5.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup

In [6]:
pip install lxml

     |████████████████████████████████| 5.5MB 17.8MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)

In [8]:
tables[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 2. Cleaning of the dataframe

In [9]:
#Droping rows where Borough = "Not assigned"
df= tables [0]
df = df[df.Borough != "Not assigned"].reset_index()
df.head()

,index,Postal Code,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


**1st Note:**
Combination of postal code no longer required. 
The Wikipedia 'List of postal codes of Canada: M' has no duplicate Postal Codes

--> df.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(','.join).reset_index()


**2nd Note:** 
The column Neighbourhood does not contain any "Not assigned" rows (see below)

In [10]:
#check if the column Neighbourhood contains any "Not assigned column"
(df['Neighbourhood'] == "Not assigned").any()

False

In [11]:
#clean dataframe drop index column

df.drop(columns=['index'])

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [12]:
#print the numbers of rows and columns of the dataframe
df.shape

(103, 4)

## Question 2

### Getting latitude and longitude

In [13]:
#importing geocoder package
import pip

In [14]:
!pip install geocoder

     |████████████████████████████████| 102kB 16.8MB/s ta 0:00:01


In [15]:
import pandas as pd

In [16]:
link = 'http://cocl.us/Geospatial_data'
df_geospatial = pd.read_csv(link)
df_geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
df_geospatial.shape

(103, 3)

In [18]:
df = df.join(df_geospatial.set_index('Postal Code'), on='Postal Code')
df

,index,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
